In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [139]:
import pandas as pd
from pathlib import Path
from pprint import pprint
from sklearn.model_selection import train_test_split
from IPython.display import JSON
from tqdm import tqdm
import json
from coprover import PROJ_ROOT, RSC_ROOT, PVSLIB_ROOT

from coprover.feats.featurize_cmdpred import *

DATA_ROOT = Path(PROJ_ROOT, "data", "pvs", "prelude")
json_files = list(DATA_ROOT.rglob("*.json"))



In [129]:

json_fpath = Path(PVSLIB_ROOT, "algebra", "cartesian_product_finite-proofs", "cartesian_product_n_add_is_union.json")
with open(json_fpath, 'r') as f:
    json_root = json.load(f)

step = json_root['proof'][8]
tagform = convert_dict2tags(step)
state_str = format_state(tagform)


In [130]:
state_str

'<ANT> <CONS> s-formula apply constant type-actual getfield constant integer s-formula apply constant type-actual apply constant apply constant type-actual constant constant apply constant constant apply constant constant <HID> '

In [131]:
def bfs_flatten(iterable):
    ret = []
    for x in iterable:
        if isinstance(x, list):
            ret.extend(bfs_flatten(x))
        elif x is None:
            continue
        else:
            ret.append(x)
    return ret

In [174]:
# Type specific converters

TAG = "tag"
TUPLE = "tuple"
LABEL = "label"
SEQUENT = "sequent"
CURRENT_GOAL = "current-goal"
ANTECEDENTS = "antecedents"
CONSEQUENTS = "consequents"
PROOFSTATE = "proofstate"

ACTUALS = "actuals"
ARGUMENT = "argument"
APPLY = "apply"
ASSERTED = "asserted?"
CONSTANT = "constant"
CONSTANT_NAME = "constant-name"
EXPRESSION = "expression"
FIELD = "field"
FORALL = "forall"
FOREACH = "foreach"
GETFIELD = "getfield"
INTEGER = "integer"
INTEGER_VALUE = "integer-value"
VARIABLE_NAME = "variable-name"

FORMULA = "formula"
SFORMULA = "s-formula"

OPERATOR = "operator"
TYPE = "type"

TRUE = "True"
FALSE = "False"
IS_ASSERTED = "isasserted"
NOT_ASSERTED = "notasserted"

INTEGER_PLACEHOLDER = "INT"

CURRENT_RULE = "current-rule"
CURRENT_INPUT = "current-input"

def get_operator(elt):
    if elt[TAG] == CONSTANT:
        return [elt[CONSTANT_NAME]]
    else:
        return process(elt)

def get_const(elt):
    """ Given a reference to a constant, returns the featurized form.
    Note: The numeric type requires we pull this from the type lookup
    """
    assert elt[TAG] == CONSTANT
    # For now, return a generic placeholder of None
    return [None]

def get_integer(elt):
    assert elt[TAG] == INTEGER
    return [elt[INTEGER_VALUE]]


import pdb
def process(elt):
    if isinstance(elt, list):
        # Special case for removing tuple indicator
        if len(elt) == 2:
            if elt[0] == TAG and elt[1] == TUPLE:
                return [None]
        return [process(item) for item in elt]
    elif isinstance(elt, str):
        return [elt]
    elif isinstance(elt, dict):
        accum = []
        tag = elt[TAG]
        if tag == PROOFSTATE:
            ants = elt[CURRENT_GOAL][ANTECEDENTS]
            cons = elt[CURRENT_GOAL][CONSEQUENTS]
            if ants is not None:
                ants = process(ants)
            if cons is not None:
                cons = process(cons)
            return [ANTECEDENTS, ants, CONSEQUENTS, cons]
        if tag == APPLY:
            accum.extend(get_operator(elt[OPERATOR]))
            accum.extend(process(elt[ARGUMENT]))
        elif tag == FORALL:
            # TODO: Get bindings with type
            accum.append(FORALL)
            accum.extend(process(elt[EXPRESSION]))
        elif tag == GETFIELD:
            accum.append(elt[FIELD])
            accum.extend(process(elt[ARGUMENT]))
        elif tag == CONSTANT:
            return get_const(elt)
        elif tag == INTEGER:
            return get_integer(elt)
        elif tag == SFORMULA:
            is_asserted = elt[ASSERTED]
            if is_asserted:
                asserted_val = IS_ASSERTED
            else:
                asserted_val = NOT_ASSERTED
            accum.extend([SFORMULA, asserted_val])
            accum.extend(process(elt[FORMULA]))
        elif isinstance(tag, list):
            # If the tag is a list, likely of item for a getfield access
            # For first pass, return empty
            return [None]
        else:
            raise Exception("Unhandled tag = {}".format(tag))
        #if accum is not None and any(["cartesian" in x for x in accum if isinstance(x, str)]):
        #    pdb.set_trace()
        return accum

In [137]:
prf = json_root['proof'][2]
print(bfs_flatten(process(prf)))
JSON(prf)

['antecedents', 'consequents', 's-formula', 'notasserted', 'forall', 'booleans__IMPLIES', 'notequal__notequal__1', 'length', 0, 'equalities__equal', 'cartesian_product_finite__cartesian_product_n', 'seq_extras__add_first', 'cartesian_product_finite__cartesian_product_set', 'cartesian_product_finite__cartesian_product_n']


<IPython.core.display.JSON object>

In [140]:
JSON(json_root)

<IPython.core.display.JSON object>

In [157]:
PROOF_SESSION = "proof-session"
PROOF = "proof"
TYPE_HASH = "type-hash"

from collections import Counter

tok_counter = Counter()

for json_fpath in tqdm(json_files):
    with open(json_fpath, 'r') as f:
        json_root_obj = json.load(f)
        if PROOF in json_root_obj:
            proof_seq = json_root_obj[PROOF]
            for idx in range(0, len(proof_seq), 2):
                state_toks = bfs_flatten(process(proof_seq[idx]))
                tok_counter.update(state_toks)

 10%|███████▎                                                               | 199/1920 [00:00<00:01, 1351.24it/s]


Exception: Unhandled tag = if

In [199]:
# Try again, this time using a strict traversal collecting tags

TABU_TAGS = set([ACTUALS, CONSTANT_NAME, LABEL,
                 TAG, TYPE, VARIABLE_NAME, CURRENT_RULE, CURRENT_INPUT])

def process2(elt, parent_key=None):
    """
    Collect all tags.  If this is a constant, check the context
    and if is an operator, collect the constant name
    """
    accum = []
    if isinstance(elt, list):
        return [process2(x) for x in elt]
    elif isinstance(elt, str):
        return [elt]
    elif isinstance(elt, int):
        return [elt]
    elif isinstance(elt, dict):
        if TAG in elt:
            accum.append(elt[TAG])
            if elt[TAG] == CONSTANT and parent_key == OPERATOR:
                accum.extend(process2(elt[CONSTANT_NAME]))
            for k, v in elt.items():
                if k not in TABU_TAGS:
                    accum.append(k)
                    accum.extend(process2(v, k))
    #if accum is not None and any([x == 'PartialFunctionDefinitions__f!1' for x in accum]):
    #    pdb.set_trace()
    return accum

In [200]:
PROOF_SESSION = "proof-session"
PROOF = "proof"
TYPE_HASH = "type-hash"

from collections import Counter

tok_counter = Counter()
to_examine_json = []

for json_fpath in tqdm(json_files):
    with open(json_fpath, 'r') as f:
        json_root_obj = json.load(f)
        if json_root_obj[TAG] == PROOF_SESSION and PROOF in json_root_obj:
            proof_seq = json_root_obj[PROOF]
            if proof_seq is not None:
                for idx in range(0, len(proof_seq), 2):
                    proof_step = proof_seq[idx]
                    state_toks = bfs_flatten(process2(proof_step))
                    #if any([x == "PartialFunctionDefinitions__f!1" for x in state_toks]):
                    #    to_examine_json.append(proof_step)
                    tok_counter.update(state_toks)

100%|███████████████████████████████████████████████████████████████████████| 1920/1920 [00:07<00:00, 265.60it/s]


In [196]:
# Examine inputs
for json_fpath in tqdm(json_files):
    with open(json_fpath, 'r') as f:
        json_root_obj = json.load(f)
        if PROOF in json_root_obj:
            proof_seq = json_root_obj[PROOF]
            if proof_seq is not None:
                print("- - - - -")
                for idx in range(1, len(proof_seq), 2):
                    input_step = proof_seq[idx]
                    print(idx, input_step)

  7%|█████▏                                                                 | 139/1920 [00:00<00:01, 1384.69it/s]

- - - - -
1 ['input', 'subtype-tcc']
- - - - -
1 ['input', 'skosimp*']
3 ['input', 'apply-extensionality', 'hide?', 't']
5 ['input', 'grind']
- - - - -
1 ['input', 'subtype-tcc']
- - - - -
1 ['input', 'skosimp*']
3 ['input', 'apply-extensionality', 'hide?', 't']
5 ['input', 'apply-extensionality', 'hide?', 't']
7 ['input', 'grind']
9 ['input', 'apply-extensionality', 'hide?', 't']
11 ['input', 'grind']
13 ['input', 'typepred', {'tag': 'constant', 'constant-name': 'PartialFunctionComposition__x!1', 'actuals': None, 'type': 1942522179}]
15 ['input', 'grind']
- - - - -
1 ['input', 'skolem', '1', ['l!1', 'p!1', 'x!1', 'y!1']]
3 ['input', 'skolem!', '*', 'keep-underscore?', None, 'skolem-typepreds?', None, 'dont-simplify?', None]
5 ['input', 'skosimp']
7 ['input', 'replace', '-1']
9 ['input', 'expand', 'length', '2', '2']
11 ['input', 'simplify', '*', 't', 't', 'rewrite-flag', None, 'flush?', None, 'linear?', None, 'cases-rewrite?', None, 'type-constraints?', 't', 'ignore-prover-output?', N

 14%|██████████▍                                                             | 278/1920 [00:00<00:03, 453.19it/s]

3 ['input', 'lemma', 'both_sides_times_pos_lt1']
5 ['input', 'inst', '-1', 'px!1 * py!1', 'pz!1 / px!1', 'pz!1 / py!1']
7 ['input', 'simplify', '*', 't', 't', 'rewrite-flag', None, 'flush?', None, 'linear?', None, 'cases-rewrite?', None, 'type-constraints?', 't', 'ignore-prover-output?', None, 'let-reduce?', 't', 'quant-simp?', None, 'implicit-typepreds?', None, 'ignore-typepreds?', None]
9 ['input', 'assert']
11 ['input', 'grind']
13 ['input', 'case', {'tag': 'apply', 'operator': {'tag': 'constant', 'constant-name': 'equalities__equal', 'actuals': [{'tag': 'type-actual', 'type': 2910104866}], 'type': 2895152588}, 'argument': [['tag', 'tuple'], {'tag': 'apply', 'operator': {'tag': 'constant', 'constant-name': 'number_fields__times', 'actuals': None, 'type': 3936226261}, 'argument': [['tag', 'tuple'], {'tag': 'apply', 'operator': {'tag': 'constant', 'constant-name': 'number_fields__divide', 'actuals': None, 'type': 3889784157}, 'argument': [['tag', 'tuple'], {'tag': 'constant', 'constan

 22%|███████████████▊                                                        | 420/1920 [00:00<00:02, 501.09it/s]

...

 30%|█████████████████████▌                                                  | 575/1920 [00:01<00:03, 342.78it/s]

1 ['input', 'skosimp*']
3 ['input', 'lemma', 'floor_div', ['x', 'x!1', 'py', {'tag': 'integer', 'integer-value': 1}, 'i', {'tag': 'integer', 'integer-value': 0}]]
5 ['input', 'replace', '-1', '1']
7 ['input', 'simplify', '*', 't', 't', 'rewrite-flag', None, 'flush?', None, 'linear?', None, 'cases-rewrite?', None, 'type-constraints?', 't', 'ignore-prover-output?', None, 'let-reduce?', 't', 'quant-simp?', None, 'implicit-typepreds?', None, 'ignore-typepreds?', None]
9 ['input', 'assert']
- - - - -
1 ['input', 'subtype-tcc']
- - - - -
1 ['input', 'grind']
- - - - -
1 ['input', 'skolem', '1', ['x!1', 'y!1']]
3 ['input', 'skolem!', '*', 'keep-underscore?', None, 'skolem-typepreds?', None, 'dont-simplify?', None]
5 ['input', 'skosimp']
7 ['input', 'expand', 'fractional']
9 ['input', 'lemma', 'floor_plus_int']
11 ['input', 'inst', '-1', '-1 * floor(x!1) - floor(y!1)', 'x!1 + y!1']
13 ['input', 'replace', '-1']
15 ['input', 'simplify', '*', 't', 't', 'rewrite-flag', None, 'flush?', None, 'line

 33%|███████████████████████▊                                                | 636/1920 [00:01<00:03, 332.38it/s]

- - - - -
1 ['input', 'skolem', '1', ['inj!1', 'S!1']]
3 ['input', 'skolem-typepred']
5 ['input', 'lemma', 'card_image', None]
7 ['input', 'use', 'card_image']
9 ['input', 'lemma', 'injection_n_to_m_var']
11 ['input', 'inst', '-', 'card(image(inj!1, S!1))', 'card(S!1)']
13 ['input', 'simplify', '*', 't', 't', 'rewrite-flag', None, 'flush?', None, 'linear?', None, 'cases-rewrite?', None, 'type-constraints?', 't', 'ignore-prover-output?', None, 'let-reduce?', 't', 'quant-simp?', None, 'implicit-typepreds?', None, 'ignore-typepreds?', None]
15 ['input', 'assert']
17 ['input', 'case', {'tag': 'exists', 'bindings': [{'tag': ['variable'], 'type': 504203496, 'variable-name': 'f'}], 'expression': {'tag': 'apply', 'operator': {'tag': 'constant', 'constant-name': 'functions__injectivep', 'actuals': [{'tag': 'type-actual', 'type': 3190075457}, {'tag': 'type-actual', 'type': 2713986665}], 'type': 3622798552}, 'argument': {'tag': ['variable'], 'type': 504203496, 'variable-name': 'f'}}}]
19 ['input'

KeyboardInterrupt: 

In [189]:
tok_counter

Counter({'proofstate': 16071,
         'current-goal': 16071,
         'sequent': 16071,
         'antecedents': 16071,
         'consequents': 16071,
         's-formula': 86048,
         'new?': 86048,
         'formula': 86048,
         'forall': 37188,
         'bindings': 55674,
         'variable': 218017,
         'expression': 55403,
         'apply': 403862,
         'operator': 403380,
         'getfield': 5996,
         'field': 5996,
         'dom': 34,
         'argument': 418689,
         'fun': 33,
         'asserted?': 86048,
         'hidden': 16071,
         'constant': 536207,
         'equalities__equal': 45881,
         'tag': 214499,
         'tuple': 214499,
         'PartialFunctionDefinitions__SPartFun_to_LPartFun': 20,
         'PartialFunctionComposition__oh__2': 9,
         'PartialFunctionComposition__oh__1': 10,
         'booleans__NOT': 59313,
         'booleans__IMPLIES': 21100,
         'PartialFunctionDefinitions__LPartFun_to_SPartFun': 30,
         'b

In [190]:
len(to_examine_json)

3

In [191]:
JSON(to_examine_json[0])

<IPython.core.display.JSON object>